In [110]:
import numpy as np
import sys    

In [111]:
class Carta:
    def __init__(self, seme, valore):
        self.seme = seme
        self.valore = valore
    
    def __str__(self):
        return str(self.valore) + " di " + self.seme
    
    def punti(self):
        if self.valore == 1: return 11
        if self.valore == 3: return 10
        if self.valore >= 8: return self.valore - 6
        return 0

class Mazzo:
    semi = ["denara", "bastoni", "coppe", "spade"]
    valori = [1,2,3,4,5,6,7,8,9,10]

    def __init__(self):
        self.carte = []
        for seme in Mazzo.semi:
            for valore in Mazzo.valori:
                carta = Carta(seme, valore)
                self.carte.append(carta)
    
    def carteRimaste(self):
        return len(self.carte)
    
    def pesca(self):
        if self.carteRimaste() == 0:
            return None
        if self.carteRimaste() == 1:
            return self.carte.pop()
        n = np.random.randint(0, self.carteRimaste())
        return self.carte.pop(n)

In [112]:
class Giocatore:
    def __init__(self, cartaInFondo):
        self.punti = 0
        self.mano = []
        self.cartaInFondo = cartaInFondo
        # altre features
        self.briscoleUscite1to7 = 0
        self.briscoleUscite8to10 =0
        self.carichiUsciti = dict()
        for seme in Mazzo.semi:
            self.carichiUsciti[seme] = 0
    
    def tira(self, cartaSulTavolo):
        print("Mano: ",self.mano[0], end=" ")
        for carta in self.mano[1:]:
            print("---",carta, end = " ")
        print()
        if cartaSulTavolo != None:
            print("Carta tirata dal primo giocatore:", cartaSulTavolo)
        sys.stdout.flush()
        numeroCarta = int(input())
        return self.mano.pop(numeroCarta)
    
    def aggiornaPunti(self, carta1, carta2):
        self.punti += carta1.punti()
        self.punti += carta2.punti()
        self.printStats()

    def aggiornaStats(self, carta1, carta2, briscola):
        # carta 1
        if carta1.seme == briscola:
            if carta1.valore <= 7: self.briscoleUscite1to7 += 1
            else: self.briscoleUscite8to10 += 1
        if carta1.valore == 1 or carta1.valore == 3:
            self.carichiUsciti[carta1.seme] += 1
        # carta 2
        if carta2.seme == briscola:
            if carta2.valore <= 7: self.briscoleUscite1to7 += 1
            else: self.briscoleUscite8to10 += 1
        if carta2.valore == 1 or carta2.valore == 3:
            self.carichiUsciti[carta2.seme] += 1

    def aggiungiInMano(self, carta):
        self.mano.append(carta)

    def printStats(self):
        print("tot briscole uscite: ", self.briscoleUscite1to7 + self.briscoleUscite8to10)
        carichiUsciti = 0
        for seme in Mazzo.semi:
            carichiUsciti += self.carichiUsciti[seme]
        print("totale carichi usciti:", carichiUsciti)
        print("punti", self.punti, "\n")

class Briscola:
    def __init__(self):
        self.mazzo = Mazzo()
        self.cartaInFondo = self.mazzo.pesca()
        self.briscola = self.cartaInFondo.seme

        self.giocatore0 = Giocatore(self.cartaInFondo)
        self.giocatore1 = Giocatore(self.cartaInFondo)

        self.giocatori = [self.giocatore0, self.giocatore1]

        for _ in range(3):
            self.giocatore0.mano.append(self.mazzo.pesca())
            self.giocatore1.mano.append(self.mazzo.pesca())

        self.cartaTirata = None
        self.ordine = [0,1]
        if np.random.randint(0,2) == 1: self.ordine.reverse()
    
    def vincitore(self):
        if self.giocatore0.punti > 60: return self.giocatore0
        if self.giocatore1.punti > 60: return self.giocatore1
        return None

    def turno(self):
        primo = self.giocatori[self.ordine[0]]
        secondo = self.giocatori[self.ordine[1]]
        if primo == self.giocatore0: print("Turno del giocatore 0")
        else: print("Turno del giocatore 1")
        cartaPrimo = primo.tira(None)

        print()
        if secondo == self.giocatore0: print("Turno del giocatore 0")
        else: print("Turno del giocatore 1")
        cartaSecondo = secondo.tira(cartaPrimo)
        cartaVincente = self.cartaVincente(cartaPrimo, cartaSecondo)
        if cartaPrimo == cartaVincente:
            vincenteTurno = primo
            print("\nmano vinta dal primo")
        else:
            vincenteTurno = secondo
            print("\nmano vinta dal secondo")
            self.ordine.reverse()
        vincenteTurno.aggiornaPunti(cartaPrimo, cartaSecondo)
        primo.aggiornaStats(cartaPrimo, cartaSecondo, self.briscola)
        secondo.aggiornaStats(cartaPrimo, cartaSecondo, self.briscola)

    def fasePescata(self):
        pescata = self.mazzo.pesca()
        self.giocatori[self.ordine[0]].aggiungiInMano(pescata)
        pescata = self.mazzo.pesca()
        if pescata == None:
            self.giocatori[self.ordine[1]].aggiungiInMano(self.cartaInFondo)
        else: self.giocatori[self.ordine[1]].aggiungiInMano(pescata)
    
    def cartaVincente(self, carta1, carta2):
        if carta1.seme == carta2.seme:
            if carta1.valore > carta2.valore: return carta1
            return carta2
        if carta1.seme == self.briscola: return carta1
        if carta2.seme == self.briscola: return carta2
        return carta1
    
    
    def partitaFinita(self):
        return (self.giocatore0.punti>60) or (self.giocatore1.punti>60) or ()
    
    def gioca(self):
        print("Carta in fondo:", self.cartaInFondo, "\n\n")
        while self.mazzo.carteRimaste()!=0:
            print("Carte rimaste nel mazzo:", self.mazzo.carteRimaste()+1)
            self.turno()
            self.fasePescata()
        self.turno()
        self.turno()
        self.turno()
        print()
        print("punti giocatore 0", self.giocatore0.punti)
        print("punti giocatore 1", self.giocatore1.punti)

In [113]:
partita = Briscola()
partita.gioca()

Carta in fondo: 4 di spade 


Carte rimaste nel mazzo: 34
Turno del giocatore 0
Mano:  6 di spade --- 7 di denara --- 1 di spade 

Turno del giocatore 1
Mano:  4 di coppe --- 5 di coppe --- 9 di denara 
Carta tirata dal primo giocatore: 7 di denara

mano vinta dal primo
tot briscole uscite:  0
totale carichi usciti: 0
punti 0 

Carte rimaste nel mazzo: 32
Turno del giocatore 0
Mano:  6 di spade --- 1 di spade --- 10 di bastoni 

Turno del giocatore 1
Mano:  4 di coppe --- 9 di denara --- 2 di bastoni 
Carta tirata dal primo giocatore: 1 di spade

mano vinta dal primo
tot briscole uscite:  0
totale carichi usciti: 0
punti 14 

Carte rimaste nel mazzo: 30
Turno del giocatore 0
Mano:  6 di spade --- 10 di bastoni --- 10 di denara 

Turno del giocatore 1
Mano:  4 di coppe --- 2 di bastoni --- 7 di bastoni 
Carta tirata dal primo giocatore: 10 di bastoni

mano vinta dal primo
tot briscole uscite:  1
totale carichi usciti: 1
punti 18 

Carte rimaste nel mazzo: 28
Turno del giocatore 0
Mano: 

In [ ]:
from sklearn.neural_network import MLPClassifier
agent = MLPClassifier(hidden_layer_sizes=(256,512),
                      activation='tanh',
                      solver='adam',
                      warm_start=True,max_iter=1
                     )